# 5.2 基于语音控制的无人机飞行（上传语音文件）

In [ ]:
import os

#构建新的提示词，知识库写入aisim_lession26.txt
kg_promot_file = "prompts/aisim_lession52.txt"

kg_prompt = """
以下是您可以用来命令无人机的一些功能。

aw.takeoff() - 起飞无人机。
aw.land() - 无人机着陆。
aw.get_drone_position() - 以与 X、Y、Z 坐标相对应的 3 个浮点数的列表形式返回无人机的当前位置。
aw.fly_to([x, y, z]) - 将无人机飞到指定的位置，该位置由对应于 X、Y、Z 坐标的三个参数列表指定。
aw.get_position(object_name)：采用字符串作为输入，指示感兴趣对象的名称，并返回 3 个浮点数的列表，指示其 X、Y、Z 坐标。

场景中存在以下对象，您将使用这些确切的名称来引用它们：

风力发电机1、风力发电机2、太阳能电池板、汽车、人群、塔1、塔2、塔3。

请注意，在函数调用的时候，请使用对应的英文名称，具体对应如下：
turbine1 ：风力发电机1
turbine2 ：风力发电机2
solarpanels：太阳能电池板
car：汽车
crowd:人群
tower1:塔1
tower2:塔2
tower3:塔3
House1:白房子
RectLight2_3:灯3
Stairs:楼梯

例如，你需要获得风力发电机1的位置，你可以这样写：
aw.get_position("turbine1")
而不是：
aw.get_position("风力发电机1")

就坐标轴约定而言，我们使用的NED，也就是"北东地"坐标系，
，即+X是北，+Y是东，+Z是下。这意味着Z值越高越负，如果原点在地面上，z值是零，地面以上是负的！向上飞是z轴减去一个值

除了无人机本身之外，所有物体都不可移动。请记住，有两个涡轮机和三个塔。当有多个相同类型的对象时，
如果我没有明确指定我指的是哪个对象，您应该始终要求我进行澄清。永远不要做出假设。

回复均按照如下例子的格式： 
```python
i=1#这里输出python代码
```
此代码功能为赋值


你无需考虑aw的import问题，在环境中已经声明
"""
os.makedirs(os.path.dirname(kg_promot_file), exist_ok=True)
pt_file = open(kg_promot_file, "w", encoding="utf-8")
pt_file.write(kg_prompt)
pt_file.close()

In [ ]:
import airsim_agent
my_agent = airsim_agent.AirSimAgent(knowledge_prompt="prompts/aisim_lession52.txt")

In [ ]:
command = "起飞"
python_code = my_agent.process(command, True) #执行代码
print("python_code: \n", python_code)

In [ ]:
command = "以楼梯为中心点，绕楼梯飞行一周，避免靠近楼梯10米的范围以内，有障碍物阻挡，完成后文字输出执行状态保存到本地当前文件夹log.txt，并避免中文乱码，内容包括运行消耗时间（保留2位小数）并在前面加上标志词，任务完成：          "
python_code = my_agent.process(command, True) #执行代码
print("python_code: \n", python_code)

In [ ]:
import sys
sys.path.append('../external-libraries')
import airsim
import airsim_wrapper
import math
import time

# 记录开始时间
start_time = time.time()
aw = airsim_wrapper.AirSimWrapper()
try:
    # 起飞
    aw.takeoff()

    # 获取楼梯位置
    stairs_pos = aw.get_position("Stairs")

    # 设定绕飞半径（大于 10 米）
    radius = 12
    # 设定飞行高度
    flight_height = -5

    # 设定绕飞点数
    num_points = 8

    for i in range(num_points):
        # 计算当前点角度
        angle = 2 * math.pi * i / num_points
        # 计算目标点坐标
        x = stairs_pos[0] + radius * math.cos(angle)
        y = stairs_pos[1] + radius * math.sin(angle)
        z = stairs_pos[2] + flight_height

        # 飞到目标点
        aw.fly_to([x, y, z])

    # 着陆
    aw.land()
    status = "成功完成"
except Exception as e:
    status = f"失败，原因: {str(e)}"

# 记录结束时间
end_time = time.time()
# 计算消耗时间，保留 2 位小数
elapsed_time = round(end_time - start_time, 2)

log_message = f"process finished status：绕楼梯飞行任务{status}，运行消耗时间: {elapsed_time} 秒"

# 将状态信息保存到本地当前文件夹的 log.txt 文件中，指定编码为 UTF - 8 避免中文乱码
with open('log.txt', 'w', encoding='utf-8') as f:
    f.write(log_message)


## 识别语音，转化为语言模型写代码的命令




In [ ]:
import recognition_module
#文字命令，录制成语音并上传对象存储
file_url = "https://yt-shanghai.tos-cn-shanghai.volces.com/mp3%E8%AF%AD%E9%9F%B3/comand.mp3"
text = recognition_module.process_mp3(file_url)
print(text)




执行mp3中的语音识别，并执行代码

In [ ]:
command = text
python_code = my_agent.process(command, True) #执行代码
print("python_code: \n", python_code)

读取本地文本，生成mp3语音

In [11]:
     # 读取本地文本log.txt,转成字符串
import voice_module
text = open("log.txt", "r", encoding="utf-8").read()
print(text)
voice_module.process_text2mp3(text)


process finished status：绕楼梯飞行任务成功完成，运行消耗时间: 22.63 秒
resp body: 
{'reqid': '09434882-10f7-4f97-afdd-24bc9b7f7bd2', 'code': 3000, 'operation': 'query', 'message': 'Success', 'sequence': -1, 'data': '...', 'addition': {'description': '[{"duration":7665,"pinline":"P R AA1 . S EH2 S  / F IH1 . N IH0 SH T  / S T AE1 . T AH0 S / sp rao4 lou2 ti1 fei1 xing2 ren4 wu5 cheng2 gong1 wan2 cheng2 sp yvn4 xing2 xiao1 hao4 shi2 jian1 sp er4 shi2 er4 dian3 liu4 san1 miao3","posline":"process/NN finished/VBD status/NN : 绕/VV 楼梯/NN 飞行/NN 任务/NN 成功/AD 完成/VV ， 运行/VV 消耗/NN 时间/NN : 二十二/CD 点/M 六三/CD 秒/M","psdline":"process#1 finished#1 status#3:绕#1楼梯#1飞行#1任务#2成功#1完成#3，运行#1消耗#1时间#3:二十#1二点#1六三秒#4","ssml_json":null,"style_name":"","tn_line":"input text:process finished status:绕楼梯飞行任务成功完成，运行消耗时间: 22.63 秒<split>high pripority rule result:process finished status:绕楼梯飞行任务成功完成，运行消耗时间: 22.63 秒<split>sub_string:22.63, pattern:A_Read_No_Zero, after_pattern:二十二点六三, nntn result:process finished status:绕楼梯飞行任务成功完成，运行消耗时间: 二十